In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from tqdm.notebook import trange, tqdm

In [ ]:
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Read Data

In [ ]:
data = pd.read_excel('rawdata9.xlsx')
print(len(data.columns))
data.describe()

In [ ]:
data.info()

In [ ]:
plt.style.use('ggplot')
data.hist(bins=25, figsize=(40,30));

# Output 변수 탐색

In [ ]:
data[['DS']].hist(bins=50, figsize=(10,10))
plt.show()

In [ ]:
threshold = range(-20, 20)
train_acc = []
train_auc = []
test_acc = []
test_auc = []

for i in tqdm(range(-20,20)):
    arr = np.array([i])

    data['DS(cat)'] = arr.searchsorted(data['DS'])

    X = data.drop(['TS', 'DS','DS(cat)'], axis=1)
    X = preprocessing.StandardScaler().fit_transform(X)
    y = data['DS(cat)']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=2)

    clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)
    
    scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')
    train_acc.append(scores.mean())
    
    scores = cross_val_score(clf, X_train, y_train, cv=3,scoring='roc_auc')
    train_auc.append(scores.mean())

    y_pred = clf.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))

    y_score = clf.predict_proba(X_test)
    test_auc.append(roc_auc_score(y_test, y_score[:, 1]))

In [ ]:
plt.rcParams["figure.figsize"] = (20,15)
df = pd.DataFrame()
# df['train acc'] = train_acc
df['train auc'] = train_auc
# df['test acc'] = test_acc
df['test auc'] = test_auc
df['threshold'] = threshold
df.set_index('threshold', drop=True, inplace=True)
df.plot.line()

# Output 변수 categorization

In [ ]:
arr = np.array([-10, 10])

data['exclude'] = arr.searchsorted(data['DS'])

In [ ]:
data = data[data.exclude != 1]

In [ ]:
data.drop(['exclude'], axis=1, inplace=True)

In [ ]:
arr = np.array([0])

data['DS(cat)'] = arr.searchsorted(data['DS'])

X = data.drop(['TS', 'DS(cat)', 'DS'], axis=1)
X = pd.DataFrame(preprocessing.StandardScaler().fit_transform(X), columns=list(X))
y = data['DS(cat)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

clf = GradientBoostingClassifier()
clf.fit(X_train,y_train)

# Feature selection

### RFE

In [ ]:
'''from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

rfecv = RFECV(estimator = clf, step = 1, cv = StratifiedKFold(3), scoring = 'roc_auc', n_jobs = -1)

rfecv.fit(X_train, y_train)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features")
plt.ylabel("AUROC")
plt.title('Recursive Feature Elimination')
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()'''

### Sequential selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.model_selection import StratifiedKFold

start = 1
end = 22

k_fold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

### Sequential Forward Selection

In [ ]:
sfs = SequentialFeatureSelector(clf,
           k_features=(start, end - 1),
           forward=True,
           floating=False,
           verbose=0,
           scoring='roc_auc',
           cv=k_fold,
           n_jobs=-1)

sfs.fit(X_train, y_train)

index_total = []
index_num_total = []
auc_total = []

for i in range(start, end):
    index = [list(X)[i] for i in sfs.subsets_[i]['feature_idx']]
    index_num = list(sfs.subsets_[i]['feature_idx'])
    index_total.append(index)
    index_num_total.append(index_num)
    auc_total.append(sfs.subsets_[i]['avg_score'])

pd.set_option('max_colwidth', 500)
pd.DataFrame(zip(auc_total, index_total, index_num_total), index=range(start, end), columns=['auc', 'features', 'col_no'])

In [ ]:
fig1 = plot_sfs(sfs.get_metric_dict(),
                kind='std_dev',
                figsize=(6, 4))

plt.ylim([0.6, 1])
plt.title('Sequential Forward Selection (95% CI)')
plt.ylabel('AUROC')
save_fig("Sequential Forward Selection")
plt.show()

# Select Features

In [ ]:
X_train_original = X_train
X_test_original = X_test

In [ ]:
index = [0,1,5]
print([list(X_train_original)[i] for i in index])
X_train = X_train_original.iloc[:, index]
X_test = X_test_original.iloc[:, index]

# ROC curve (internal)

In [ ]:
from sklearn.metrics import roc_curve, auc
from numpy import interp
from sklearn.model_selection import StratifiedKFold

skfolds = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

thresholds = []
tprs = []
aucs = []
base_fpr = np.linspace(0, 1, 101)

for train, test in skfolds.split(X_train, y_train):
    
    model = clf.fit(X_train.iloc[train], y_train.iloc[train])
    
    y_score = model.predict_proba(X_train.iloc[test])
    
    fpr, tpr, threshold = roc_curve(y_train.iloc[test], y_score[:, 1])
    print(f'len_y_true: {len(y_train.iloc[test])}, len_y_pred: {len(y_score[:, 1])}, len_fpr: {len(fpr)}, len_tpr: {len(tpr)}')
    
    roc_auc = auc(fpr, tpr)
    
    aucs.append(roc_auc)
    
    tpr = interp(base_fpr, fpr, tpr)
    tpr[0] = 0.0
    tprs.append(tpr)
    
    threshold = interp(base_fpr, fpr, threshold)
    thresholds.append(threshold)

tprs = np.array(tprs)
mean_tprs = tprs.mean(axis=0)
std = tprs.std(axis=0)

thresholds = np.array(thresholds)
mean_thresholds = thresholds.mean(axis=0)

mean_auc = auc(base_fpr, mean_tprs)
std_auc = np.std(aucs)

tprs_upper = np.minimum(mean_tprs + std, 1)
tprs_lower = mean_tprs - std

plt.style.use('ggplot')
plt.figure(figsize=(8, 8))
plt.plot(base_fpr, mean_tprs, 'b', alpha = 0.8, label = r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),)
plt.fill_between(base_fpr, tprs_lower, tprs_upper, color = 'blue', alpha = 0.2)
plt.plot([0, 1], [0, 1], linestyle = '--', lw = 2, color = 'r', alpha= 0.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
# plt.title('Receiver operating characteristic (ROC) curve')
save_fig('Figure 2B')
plt.show()

# Youden index, sensitivity, specificity

In [ ]:
youden_index = mean_tprs-base_fpr
max_index = np.argmax(youden_index)
sensitivity = mean_tprs[max_index]
specificity = 1 - base_fpr[max_index]
print("max youden index", youden_index[max_index],"thresholds", mean_thresholds[max_index], "sensitivity", sensitivity, "specificity", specificity)

# Feature importance

In [ ]:
clf.feature_importances_
feature_importance_dict = {k: v for k, v in sorted(dict(zip(X_train, clf.feature_importances_)).items(), key=lambda item: item[1], reverse=True)}

plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(8,5))
fig.align_ylabels()

feature = feature_importance_dict.keys()
y_pos = np.arange(len(feature))
feature_importance = feature_importance_dict.values()

ax.barh(y_pos, feature_importance, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(feature)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Feature importance (%)')
save_fig('Figure 2A')

# Validation

In [ ]:
y_test_score = clf.predict_proba(X_test)

fpr, tpr, threshold = roc_curve(y_test, y_test_score[:, 1])

roc_auc = auc(fpr, tpr)

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, 'b', alpha = 0.8, label = r'ROC (AUC = %0.2f)' % (roc_auc),)
plt.plot([0, 1], [0, 1], linestyle = '--', lw = 2, color = 'r', alpha= 0.8)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
#plt.title('Receiver operating characteristic (ROC) curve')
save_fig('Figure 2C')
plt.show()

In [ ]:
youden_index = tpr - fpr
max_index = np.argmax(youden_index)
sensitivity = tpr[max_index]
specificity = 1 - fpr[max_index]
print(f'max youden index: {youden_index[max_index]:.03f}, thresholds: {threshold[max_index]:.03f}, sensitivity: {sensitivity:.03f}, specificity: {specificity:.03f}')